## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [37]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [38]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [4]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [39]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [7]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [9]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [40]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

In [21]:
chain

PromptTemplate(input_variables=['how', 'topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 {how} 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x169cc9150>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x169cbb8d0>, root_client=<openai.OpenAI object at 0x169cd7710>, root_async_client=<openai.AsyncOpenAI object at 0x169cca810>, model_name='gpt-4.1-nano', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [46]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how": "간단하게"}

In [47]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명하면 다음과 같습니다.\n\n1. **데이터 수집**: 먼저 컴퓨터가 배우기 위해 많은 예제(데이터)를 모읍니다. 예를 들어, 고양이 사진과 고양이 이름이 적힌 데이터입니다.\n\n2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌처럼 작동하는 인공신경망일 수 있어요.\n\n3. **학습 과정**: 모델이 데이터를 보고 '이것이 고양이야'라고 맞추도록 시도합니다. 처음에는 틀릴 수도 있지만, 정답과 비교해서 틀린 부분을 수정하는 과정을 반복합니다. 이때 '오차'라는 것을 계산해서, 이 오차를 줄이기 위해 모델의 내부 값을 조정합니다.\n\n4. **반복과 개선**: 이 과정을 여러 번 반복하면서 모델이 점점 더 정확하게 데이터를 이해하게 됩니다.\n\n5. **완성**: 충분히 학습이 되면, 새로운 데이터(예를 들어, 새로운 고양이 사진)를 보여줬을 때도 올바르게 인식할 수 있게 됩니다.\n\n즉, 인공지능은 많은 예제와 반복 학습을 통해 '이것이 맞다'고 스스로 배우는 과정입니다!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 308, 'prompt_tokens': 25, 'total_tokens': 333, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerpr

In [51]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model
chain.invoke("인공지능 모델의 학습 원리")  # input이 하나라면 가능.

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 사진을 보고 고양이인지 강아지인지 구별하는 모델이라면, 고양이와 강아지 사진이 필요하겠죠.\n\n2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 여러 층으로 이루어진 인공신경망일 수 있어요.\n\n3. **학습 과정**:\n   - **예측하기**: 모델은 처음에는 아무것도 모르기 때문에 무작위로 예측을 해요.\n   - **오차 계산**: 그리고 그 예측이 얼마나 맞았는지(혹은 틀렸는지)를 평가해요. 이걸 '오차'라고 부릅니다.\n   - **수정하기**: 오차가 크면, 모델의 내부 값(가중치)을 조금씩 조정해서 다음에는 더 잘 맞도록 해요.\n   \n4. **반복 학습**: 이 과정을 수천, 수만 번 반복하면서 모델은 점점 더 정확하게 예측할 수 있게 돼요.\n\n5. **완성**: 충분히 학습이 되면, 모델은 새로운 데이터(예를 들어, 새로운 사진)가 들어왔을 때도 잘 구별할 수 있게 됩니다.\n\n요약하자면, 인공지능 모델은 많은 데이터를 보고, 예측을 하고, 틀린 부분을 수정하면서 점점 더 똑똑해지는 과정입니다. 이 과정을 '학습'이라고 부르죠!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 391, 'prompt_tokens': 22, 'total_tokens': 413, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt

아래는 스트리밍을 출력하는 예시 입니다.

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 출력파서(Output Parser)


In [27]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [53]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser  # 답변이 바로 출력. 원래는 AIMessage라는 객체 형태로 옴

In [54]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

"물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예시(데이터)를 모아요. 예를 들어, 고양이 사진과 강아지 사진을 모으는 거죠.\n\n2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 분류할 수 있도록 인공지능의 '뇌' 역할을 하는 모델을 만듭니다. 이 모델은 수학적인 함수와 구조로 이루어져 있어요.\n\n3. **학습 과정**:\n   - 모델이 처음에는 아무것도 모르기 때문에, 사진이 고양이인지 강아지인지 맞추지 못해요.\n   - 모델이 예측을 하고, 그 예측이 맞았는지 틀렸는지 알려주는 '정답'을 비교해요.\n   - 틀렸다면, 모델은 '오류'를 계산하고, 이 오류를 줄이기 위해 내부의 수치(가중치)를 조금씩 조정해요.\n   - 이 과정을 여러 번 반복하면서, 모델은 점점 더 정확하게 사진을 구분할 수 있게 되는 거예요.\n\n4. **최적화**: 이때 사용하는 방법이 '경사 하강법' 같은 최적화 기법으로, 오류를 최소화하는 방향으로 가중치를 조정하는 과정입니다.\n\n5. **학습 완료**: 여러 번 반복해서 학습하면, 모델은 새로운 사진이 들어왔을 때도 고양이인지 강아지인지 잘 맞출 수 있게 돼요.\n\n요약하자면, 인공지능은 많은 데이터를 보고, 예측을 하고, 틀리면 수정하는 과정을 반복하면서 점점 더 똑똑해지는 거예요!"

In [30]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model

In [32]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진과 강아지 사진을 많이 모으는 거죠.\n\n2. **모델 설계**: 그 다음, 컴퓨터가 데이터를 이해하고 학습할 수 있도록 '모델'이라는 구조를 만듭니다. 이 모델은 사람의 뇌처럼 여러 연결된 작은 부분(뉴런과 비슷한 구조)을 가지고 있어요.\n\n3. **학습 과정**:\n   - 모델은 처음에는 아무것도 모르기 때문에, 예제 데이터를 보고 '이것이 고양이일까, 강아지일까?'라고 추측해요.\n   - 그리고 정답(예를 들어, 이 사진은 고양이야!)과 비교해서 틀렸다면, 틀린 정도를 계산해요.\n   - 그 차이를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 조정해요. 이 과정을 여러 번 반복하면서 점점 더 정확하게 사진을 구별하게 되는 거죠.\n\n4. **최적화**: 이 과정을 통해 모델이 점점 더 정답에 가까워지도록 조정하는 것이 '학습'입니다. 이때 사용하는 방법이 '경사 하강법' 같은 최적화 기법이에요.\n\n5. **검증과 테스트**: 학습이 끝나면, 새로운 데이터(보지 않은 예제)를 넣어 모델이 잘 맞추는지 확인해요. 이렇게 해서 모델이 실제로 잘 작동하는지 검증하는 거죠.\n\n요약하자면, 인공지능 모델은 많은 데이터를 보고, 정답과 비교하며, 틀린 부분을 수정하는 과정을 반복해서 점점 더 똑똑해지는 거예요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 415, 'prompt_tokens': 22, 'total_tokens': 437, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens

In [31]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 사진을 보고 고양이인지 강아지인지 구별하는 모델이라면, 고양이와 강아지 사진이 필요하겠죠.

2. **모델 설계**: 그 다음, 인공지능이 어떤 방식으로 정보를 처리할지 정하는 '모델'이라는 구조를 만듭니다. 이 구조는 사람의 뇌와 비슷하게 여러 '뉴런'이 연결된 네트워크처럼 생겼어요.

3. **학습 과정**:
   - 모델은 처음에는 아무것도 모르기 때문에, 예제 데이터를 보고 예측을 해봅니다.
   - 그리고 실제 정답(예를 들어, 사진이 고양이인지 강아지인지)을 알려주면, 모델이 만든 예측이 얼마나 맞았는지 평가합니다.
   - 맞지 않은 경우, '오차'라는 차이를 계산하고, 이 오차를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 조정합니다. 이 과정을 '반복'하면서 모델이 점점 더 정확하게 예측하게 되는 것이죠.

4. **반복 학습**: 이 과정을 여러 번 반복하면서, 모델은 점점 더 데이터를 잘 이해하고, 새로운 데이터에 대해서도 올바른 예측을 할 수 있게 됩니다.

요약하자면, 인공지능 모델은 많은 데이터를 보고, 예측을 하고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정입니다. 이렇게 해서 우리가 원하는 작업을 잘 수행할 수 있게 되는 거예요!

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [55]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""  # 다음 줄 실행하려면 """
# 중괄호 두 개면 그냥 값이 됨.

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [57]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [58]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  - Waiter: "Good evening! How many people are in your party?"
  - You: "Just two, please."
  - Waiter: "Great! Here are the menus. Can I get you started with something to drink?"
  - You: "Yes, I’d like a glass of water and my friend will have a cola."
  - Waiter: "Perfect! And are you ready to order your meals?"
  - You: "Yes, I’d like the grilled chicken salad, and my friend will have the cheeseburger with fries."
  - Waiter: "Sounds good! I’ll get that started for you. Anything else?"
  - You: "No, that’s all for now. Thank you!"

- 한글 해석:
  - 웨이터: "안녕하세요! 몇 명이신가요?"
  - 당신: "두 명입니다."
  - 웨이터: "좋습니다! 여기 메뉴입니다. 음료수는 무엇을 드릴까요?"
  - 당신: "네, 저는 물 한 잔이 필요하고 제 친구는 콜라를 마실 거예요."
  - 웨이터: "완벽합니다! 식사는 주문 준비 되셨나요?"
  - 당신: "네, 저는 그릴드 치킨 샐러드를 원하고, 제 친구는 치즈버거와 감자튀김을 주문할 거예요."
  - 웨이터: "좋은 선택입니다! 바로 시작하겠습니다. 더 필요한 것이 있나요?"
  - 당신: "아니요, 지금은 그게 전부입니다. 감사합니다!"


In [36]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hi there! How many people are in your party?  
B: Just one, please.  
A: Great! Here’s the menu. Can I get you anything to drink?  
B: Yes, I’ll have a glass of water, please.  
A: Sure! Are you ready to order?  
B: Yes, I’d like the grilled chicken with a side salad.  
A: Excellent choice! Would you like any dressings for your salad?  
B: Yes, please. I’ll have ranch dressing.  
A: Perfect! I’ll have that right out for you.

- 한글 해석:
A: 안녕하세요! 일행이 몇 분이신가요?  
B: 저 혼자입니다.  
A: 좋습니다! 여기 메뉴입니다. 음료수는 어떻게 하시겠어요?  
B: 네, 물 한 잔 주세요.  
A: 알겠습니다! 주문하실 준비가 되셨나요?  
B: 네, 그릴에 구운 닭고기와 사이드 샐러드를 주문하겠습니다.  
A: 훌륭한 선택입니다! 샐러드에 어떤 드레싱을 원하시나요?  
B: 네, 랜치 드레싱으로 주세요.  
A: 완벽합니다! 바로 준비해 드리겠습니다.

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)